In [1]:
import pandas as pd
import numpy as np
!pip install datasets

     |████████████████████████████████| 298 kB 31.8 MB/s 
     |████████████████████████████████| 243 kB 59.1 MB/s 
     |████████████████████████████████| 132 kB 51.5 MB/s 
     |████████████████████████████████| 59 kB 7.6 MB/s 
     |████████████████████████████████| 1.1 MB 40.9 MB/s 
     |████████████████████████████████| 271 kB 60.6 MB/s 
     |████████████████████████████████| 192 kB 73.9 MB/s 
     |████████████████████████████████| 160 kB 69.5 MB/s 


In [2]:
from datasets import load_dataset

dataemotion = load_dataset("go_emotions", "raw")

Downloading:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.4M [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

Dataset go_emotions downloaded and prepared to /root/.cache/huggingface/datasets/go_emotions/raw/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
#Mengubah dataset train ke format dataframe
train_df = pd.DataFrame(dataemotion['train'])
train_df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


**DATA EKSPLORASI**

In [6]:
#Menghapus atribut pada data
unuse = ['id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear']

feats_train_df = train_df.drop(columns = unuse)

#Menampilkan data yang dihapus atribut
feats_train_df.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


**PREPROCESSING DATA**

In [14]:
#Data Cleaning

#Removal of Punctuations
import string
string.punctuation
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

feats_train_df["text_wo_punct"] = feats_train_df["text"].apply(lambda text: remove_punctuation(text))
feats_train_df.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,text_wo_punct
0,That game hurt.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,That game hurt
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,sexuality shouldn’t be a grouping category It...
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,You do right if you dont care then fuck em
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Man I love reddit
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NAME was nowhere near them he was by the Falcon


In [15]:
#mengubah kalimat ke huruf kecil
feats_train_df["text_lower"] = feats_train_df["text"].str.lower()
feats_train_df.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,text_wo_punct,text_lower
0,That game hurt.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,That game hurt,that game hurt.
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,sexuality shouldn’t be a grouping category It...,>sexuality shouldn’t be a grouping category i...
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,You do right if you dont care then fuck em,"you do right, if you don't care then fuck 'em!"
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Man I love reddit,man i love reddit.
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NAME was nowhere near them he was by the Falcon,"[name] was nowhere near them, he was by the fa..."


In [16]:
#Tokenisasi
import nltk
nltk.download('punkt')

def tokenization(text):
    tokens = nltk.word_tokenize(text)
    return tokens

#tokenization pada dataset
feats_train_df['text'] = feats_train_df['text'].apply(lambda x: tokenization(x))
feats_train_df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,text_wo_punct,text_lower
0,"[That, game, hurt, .]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,That game hurt,that game hurt.
1,"[>, sexuality, shouldn, ’, t, be, a, grouping,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,sexuality shouldn’t be a grouping category It...,>sexuality shouldn’t be a grouping category i...
2,"[You, do, right, ,, if, you, do, n't, care, th...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,You do right if you dont care then fuck em,"you do right, if you don't care then fuck 'em!"
3,"[Man, I, love, reddit, .]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Man I love reddit,man i love reddit.
4,"[[, NAME, ], was, nowhere, near, them, ,, he, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NAME was nowhere near them he was by the Falcon,"[name] was nowhere near them, he was by the fa..."


In [22]:
#Stopwords Removal
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')


def stopwords_remove(inputs):
    return [item for item in inputs if item not in stopwords]

feats_train_df['text_stop'] = feats_train_df['text'].apply(stopwords_remove)
feats_train_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,text_wo_punct,text_lower,text_stop
0,"[That, game, hurt, .]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,That game hurt,that game hurt.,"[That, game, hurt, .]"
1,"[>, sexuality, shouldn, ’, t, be, a, grouping,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,sexuality shouldn’t be a grouping category It...,>sexuality shouldn’t be a grouping category i...,"[>, sexuality, ’, grouping, category, It, make..."
2,"[You, do, right, ,, if, you, do, n't, care, th...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,You do right if you dont care then fuck em,"you do right, if you don't care then fuck 'em!","[You, right, ,, n't, care, fuck, 'em, !]"
3,"[Man, I, love, reddit, .]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Man I love reddit,man i love reddit.,"[Man, I, love, reddit, .]"
4,"[[, NAME, ], was, nowhere, near, them, ,, he, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NAME was nowhere near them he was by the Falcon,"[name] was nowhere near them, he was by the fa...","[[, NAME, ], nowhere, near, ,, Falcon, .]"


In [26]:
#Stemming
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def stemming(text):
  stem_text = [stemmer.stem(word) for word in text]
  return stem_text

feats_train_df['text_stemmed'] = feats_train_df['text'].apply(lambda text: stemming(text))
feats_train_df.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,text_wo_punct,text_lower,text_stop,text_stemmed
0,"[That, game, hurt, .]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,That game hurt,that game hurt.,"[That, game, hurt, .]","[that, game, hurt, .]"
1,"[>, sexuality, shouldn, ’, t, be, a, grouping,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,sexuality shouldn’t be a grouping category It...,>sexuality shouldn’t be a grouping category i...,"[>, sexuality, ’, grouping, category, It, make...","[>, sexual, shouldn, ’, t, be, a, group, categ..."
2,"[You, do, right, ,, if, you, do, n't, care, th...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,You do right if you dont care then fuck em,"you do right, if you don't care then fuck 'em!","[You, right, ,, n't, care, fuck, 'em, !]","[you, do, right, ,, if, you, do, n't, care, th..."
3,"[Man, I, love, reddit, .]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Man I love reddit,man i love reddit.,"[Man, I, love, reddit, .]","[man, I, love, reddit, .]"
4,"[[, NAME, ], was, nowhere, near, them, ,, he, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NAME was nowhere near them he was by the Falcon,"[name] was nowhere near them, he was by the fa...","[[, NAME, ], nowhere, near, ,, Falcon, .]","[[, name, ], wa, nowher, near, them, ,, he, wa..."


In [29]:
#Lematisasi
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatization(inputs):
    return [lemmatizer.lemmatize(word=x, pos='v') for x in inputs]

feats_train_df['text_lemmatized'] = feats_train_df['text_stop'].apply(lemmatization)
feats_train_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,text_wo_punct,text_lower,text_stop,text_stemmed,text_lemmatized
0,"[That, game, hurt, .]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,That game hurt,that game hurt.,"[That, game, hurt, .]","[that, game, hurt, .]","[That, game, hurt, .]"
1,"[>, sexuality, shouldn, ’, t, be, a, grouping,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,sexuality shouldn’t be a grouping category It...,>sexuality shouldn’t be a grouping category i...,"[>, sexuality, ’, grouping, category, It, make...","[>, sexual, shouldn, ’, t, be, a, group, categ...","[>, sexuality, ’, group, category, It, make, d..."
2,"[You, do, right, ,, if, you, do, n't, care, th...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,You do right if you dont care then fuck em,"you do right, if you don't care then fuck 'em!","[You, right, ,, n't, care, fuck, 'em, !]","[you, do, right, ,, if, you, do, n't, care, th...","[You, right, ,, n't, care, fuck, 'em, !]"
3,"[Man, I, love, reddit, .]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Man I love reddit,man i love reddit.,"[Man, I, love, reddit, .]","[man, I, love, reddit, .]","[Man, I, love, reddit, .]"
4,"[[, NAME, ], was, nowhere, near, them, ,, he, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NAME was nowhere near them he was by the Falcon,"[name] was nowhere near them, he was by the fa...","[[, NAME, ], nowhere, near, ,, Falcon, .]","[[, name, ], wa, nowher, near, them, ,, he, wa...","[[, NAME, ], nowhere, near, ,, Falcon, .]"
